# Kernel Shapley Study

The KernelSHAP algorithm was introduced in
[[1]](https://arxiv.org/abs/1611.07478) and is further discussed in
[[2]](https://www.nature.com/articles/s42256-023-00657-x). It is designed to be
a fast approximation of the theoretical Shapley values definition, which usually
isn't computable when there are more than a few features. They argue that the
approximation is possible because the Shapley values are the coefficients for a
particular weighted linear regression, and by subsampling rows of that
regression, we can speed up the computation. The exact form of the weighted
linear regression is not described in very much detail, though.  The purpose of
this exercise is to study a simplified version of the [official
implementation](https://github.com/shap/shap/blob/master/shap/explainers/_kernel.py).
It gives the same output (see the plot below) but doesn't do any extra error
handling or additional computational speedups (e.g., using the normal equations
instead of `sklearn`, which spends time computing some unecessary outputs). The
weighted regression on line 43 is exactly the regression mentioned in these
papers -- the tricky part is understanding what exactly the input response and
predictors are.

To install all the packages used here, you can run:

`conda env create -f https://github.com/krisrs1128/stat992_s25/raw/refs/heads/main/demos/stat992_week3.yml`

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm

class SimpleKernelExplainer:
    def __init__(self, model, data):
        self.model = model
        self.data = data
        self.N, self.P = data.shape
        self.f_bar = np.mean(model(data))

    def shap_values(self, X, nsamples="auto"):
        if nsamples == "auto":
            nsamples = 2 * self.P + 2048

        explanations = []
        for i in tqdm(range(X.shape[0])):
            explanations.append(self.explain(X[i:(i + 1)], nsamples))
        return np.array(explanations)

    def explain(self, x, nsamples):
        M = self.P

        # initialize data structures
        mask_matrix = np.zeros((nsamples, M))
        kernel_weights = np.zeros(nsamples)
        synth_data = np.tile(self.data, (nsamples, 1))

        # sample random subsets
        for i in range(nsamples):
            mask = np.random.choice([0, 1], size=M)
            mask_matrix[i] = mask
            synth_data[i * self.N:(i + 1) * self.N, mask == 1] = x[0, mask == 1]
            kernel_weights[i] = (M - np.sum(mask)) * np.sum(mask)

        # model predictions on the synthetic data
        model_out = self.model(synth_data)
        f = np.mean(model_out.reshape(nsamples, self.N, -1), axis=1) - self.f_bar

        # weighted linear regression on adjusted f
        X = mask_matrix - mask_matrix[:, -1][:, None]
        y = f.flatten() - mask_matrix[:, -1] * (self.model(x) - self.f_bar)
        lm = LinearRegression(fit_intercept=False).fit(X, y, sample_weight=kernel_weights)

        # return shapley approximation
        phi = np.zeros(M)
        phi[:-1] = lm.coef_[:-1]
        phi[-1] = (self.model(x) - self.f_bar) - np.sum(lm.coef_)
        return phi

This block makes up a dataset and fits a random forest model for us to run our feature attribution method.

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_friedman1

X, y = make_friedman1(n_samples=100, n_features=5, noise=0.1)
model = RandomForestRegressor(n_estimators=100)
model.fit(X, y)

RandomForestRegressor()

Next we apply our simplified KernelSHAP implementation. This will take ~ 30 seconds.

In [3]:
explainer = SimpleKernelExplainer(model.predict, X)
our_shap_values = explainer.shap_values(X, nsamples=1000)

100%|██████████| 100/100 [00:22<00:00,  4.42it/s]


To understand what this code is doing, try answering:

* How does the `mask` variable relate to the subsets $S$ discussed in the notes?
* How many linear regressions are run to compute SHAP values across all samples? How large are the input datasets for each of those regressions, and how is it related to `nsamples`?
* What is the relationship between the returned attributions $\varphi$ and the coefficients of our weighted linear regression?
* `synth_data` can be viewed as `nsamples` blocks of `N` rows each. How do the blocks differ from one another?
* If we ignore the terms involving the last column of `mask_matrix` (i.e., `mask_matrix[:, -1]`), then how can we interpret the response of the weighted linear regression on line 43?

Hint: You can use Python debugger (`import pdb; pdb.set_trace()`) to pause the code at a specific line and inspect the state of all variables at that point.

## Official Implementation

Here is the official KernelSHAP function from the `shap` package.

In [4]:
import shap

explainer_kernel = shap.KernelExplainer(model.predict, X)
shap_values = explainer_kernel.shap_values(X, nsamples=1000)

100%|██████████| 100/100 [00:03<00:00, 27.17it/s]


We plot the two sets of SHAP values against one another. Each point is one sample-by-feature combination. The two approaches yield nearly the same estimates. The small differences are likely due to the subsets that happened to be sampled in each case.

In [5]:
import altair as alt
import pandas as pd

# Convert shap_values and shap_values2 to DataFrames and melt to long format
our_shap_df = pd.DataFrame(our_shap_values, columns=[[f"""feature_{s}""" for s in range(5)]]).melt(var_name='Feature', value_name='Our SHAP Value')
shap_df = pd.DataFrame(shap_values, columns=[[f"""feature_{s}""" for s in range(5)]]).melt(var_name='Feature', value_name='SHAP Value')

# Concatenate the DataFrames
shap_combined = pd.concat([our_shap_df, shap_df['SHAP Value']], axis=1)

# Create the Altair plot
alt.Chart(shap_combined)\
   .mark_point()\
   .encode(
      x='Our SHAP Value',
      y='SHAP Value',
      color='Feature'
    ).properties(title='SHAP Values Comparison')

alt.Chart(...)